In [1]:
# IMPORTAMOS LIBRERIAS
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import mysql.connector

pd.set_option('display.max_columns', None) 


In [2]:
# DEFINIR LA ESTRUCTURA DE LA BBDD
# Crear la base de datos
cnx = mysql.connector.connect(
    user="root",
    password="AlumnaAdalab",
    host="127.0.0.1",
    auth_plugin='mysql_native_password'
)

cursor = cnx.cursor()
query_create_bd = "CREATE SCHEMA IF NOT EXISTS `ABCCorporation`"

cursor.execute(query_create_bd)

In [3]:
# Crear Tablas

# Table `ABCCorporation`.`PERSONAL_INFO`
# -----------------------------------------------------

query_create_table_personal = """
CREATE TABLE IF NOT EXISTS `abccorporation`.`PERSONAL_INFO` (
  `IdEmployee` INT NOT NULL,
  `EmployeeNumber` VARCHAR(45) NULL,
  `Age` INT NULL,
  `DateBirth` INT NULL,
  `Gender` VARCHAR(45) NULL,
  `MaritalStatus` VARCHAR(45) NULL,
  `DistanceFromHome` INT NULL,
  `Education` INT NULL,
  `EducationField` VARCHAR(100) NULL,
  PRIMARY KEY (`IdEmployee`))
"""
cursor.execute(query_create_table_personal)


# Table `ABCCorporation`.`EMPLOYEE_SATISFACTION`
# ----------------------------------------

query_create_table_satisfaction = """
CREATE TABLE IF NOT EXISTS `abccorporation`.`EMPLOYEE_SATISFACTION` (
  `IdEmployeeSatisfaction` INT NOT NULL AUTO_INCREMENT,
  `EnvironmentSatisfaction` INT NULL,
  `JobInvolvement` INT NULL,
  `JobSatisfaction` INT NULL,
  `RelationshipSatisfaction` INT NULL,
  `WorkLifeBalance` INT NULL,
  PRIMARY KEY (`IdEmployeeSatisfaction`))

"""

cursor.execute(query_create_table_satisfaction)


# Table `ABCCorporation`.`JOB_DATA`
# -----------------------------------------------------

query_create_table_job = """
CREATE TABLE IF NOT EXISTS `abccorporation`.`JOB_DATA` (
  `IdJobData` INT NOT NULL AUTO_INCREMENT,
  `Attrition` VARCHAR(45) NULL,
  `JobLevel` INT NULL,
  `JobRole` VARCHAR(100) NULL,
  `NumCompaniesWorked` INT NULL,
  `TotalWorkingYears` VARCHAR(45) NULL,
  `YearsAtCompany` INT NULL,
  `YearsSinceLastPromotion` INT NULL,
  `YearsWithCurrManager` INT NULL,
  PRIMARY KEY (`IdJobData`))
"""

cursor.execute(query_create_table_job)


# Table `ABCCorporation`.`SALARY`
# -----------------------------------------------------

query_create_table_salary = """
CREATE TABLE IF NOT EXISTS `abccorporation`.`SALARY` (
  `EmployeeSalary` INT NOT NULL AUTO_INCREMENT,
  `HourlyRate` FLOAT NULL,
  `DailyRate` FLOAT NULL,
  `MonthlyRate` FLOAT NULL,
  `PercentSalaryHike` INT NULL,
  PRIMARY KEY (`EmployeeSalary`))
  
"""

cursor.execute(query_create_table_salary)

# Table `ABCCorporation`.`JOB_EXTRAS`
# -----------------------------------------------------

query_create_table_extras = """
CREATE TABLE IF NOT EXISTS `abccorporation`.`JOB_EXTRAS` (
  `IdJobExtras` INT NOT NULL AUTO_INCREMENT,
  `OverTime` VARCHAR(45) NULL,
  `BusinessTravel` VARCHAR(45) NULL,
  `StockOptionLevel` INT NULL,
  `TrainingTimesLastYear` INT NULL,
  `RemoteWork` TINYINT NULL,
  `PerformanceRating` INT NULL,
  PRIMARY KEY (`IdJobExtras`))

"""

cursor.execute(query_create_table_extras)

# Table `ABCCorporation`.`COMBINATION`
# PRIMARY KEY (`IdComb`, `PERSONAL_INFO_IdEmployee`, `EMPLOYEE_SATISFACTION_IdEmployeeSatisfaction`, `JOB_DATA_IdJobData`, `SALARY_EmployeeSalary`, `JOB_EXTRAS_IdJobExtras`),
# -----------------------------------------------------
#   `IdComb` INT NOT NULL AUTO_INCREMENT,
#  `PERSONAL_INFO_IdEmployee` INT NOT NULL,
#  `EMPLOYEE_SATISFACTION_IdEmployeeSatisfaction` INT NOT NULL,
##  `JOB_DATA_IdJobData` INT NOT NULL,
#  `SALARY_EmployeeSalary` INT NOT NULL,
 # `JOB_EXTRAS_IdJobExtras` INT NOT NULL,

query_create_table_combination = """
CREATE TABLE IF NOT EXISTS `abccorporation`.`COMBINATION` (
  `IdComb` INT NOT NULL AUTO_INCREMENT,
  `IdEmployee` INT NOT NULL,
  `IdEmployeeSatisfaction` INT NOT NULL,
  `IdJobData` INT NOT NULL,
  `EmployeeSalary` INT NOT NULL,
  `IdJobExtras` INT NOT NULL,
  PRIMARY KEY (`IdComb`),
  INDEX `fk_COMBINATION_PERSONAL_INFO_idx` (`IdEmployee` ASC) VISIBLE,
  INDEX `fk_COMBINATION_EMPLOYEE_SATISFACTION_idx` (`IdEmployeeSatisfaction` ASC) VISIBLE,
  INDEX `fk_COMBINATION_JOB_DATA_idx` (`IdJobData` ASC) VISIBLE,
  INDEX `fk_COMBINATION_SALARY_idx` (`EmployeeSalary` ASC) VISIBLE,
  INDEX `fk_COMBINATION_JOB_EXTRAS_idx` (`IdJobExtras` ASC) VISIBLE,
  CONSTRAINT `fk_COMBINATION_PERSONAL_INFO`
    FOREIGN KEY (`IdEmployee`)
    REFERENCES `abccorporation`.`PERSONAL_INFO` (`IdEmployee`)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT `fk_COMBINATION_EMPLOYEE_SATISFACTION`
    FOREIGN KEY (`IdEmployeeSatisfaction`)
    REFERENCES `abccorporation`.`EMPLOYEE_SATISFACTION` (`IdEmployeeSatisfaction`)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT `fk_COMBINATION_JOB_DATA`
    FOREIGN KEY (`IdJobData`)
    REFERENCES `abccorporation`.`JOB_DATA` (`IdJobData`)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT `fk_COMBINATION_SALARY`
    FOREIGN KEY (`EmployeeSalary`)
    REFERENCES `abccorporation`.`SALARY` (`EmployeeSalary`)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT `fk_COMBINATION_JOB_EXTRAS`
    FOREIGN KEY (`IdJobExtras`)
    REFERENCES `abccorporation`.`JOB_EXTRAS` (`IdJobExtras`)
    ON DELETE CASCADE
    ON UPDATE CASCADE);
"""

cursor.execute(query_create_table_combination)

cnx.close()


In [4]:
# Leer archivo CSV en un DataFrame
df = pd.read_csv('files/df_fase1.csv')


In [5]:
# Funcion para convertir a entero
def convertir(x): 
    
    # Convertir solo las columnas de tipo int64
    lista = []
 
    for tupla in x:
        lista_intermedia = []
    
        for elem in tupla:
            try:
                lista_intermedia.append(int(elem))
            except:
                lista_intermedia.append(elem)
        
        lista.append(tuple(lista_intermedia))
        
    return lista

In [6]:
# Insertar datos

cnx = mysql.connector.connect(
    user="root",
    password="AlumnaAdalab",
    host="127.0.0.1",
    auth_plugin='mysql_native_password'
)

cursor = cnx.cursor()

### INSERCCION PERSONAL_INFO

query_insertar_tablapersonal = """ INSERT INTO `ABCCorporation`.`PERSONAL_INFO` (`IdEmployee`, `EmployeeNumber`, `Age`, `DateBirth`, `Gender`, `MaritalStatus`, `DistanceFromHome`, `Education`, `EducationField`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""

datos_tabla_personal = list(set(zip
                                (df["IdEmployee"].values, 
                                 df["EmployeeNumber"].values,
                                 df["Age"].values,
                                 df["DateBirth"].values,
                                 df["Gender"].values,
                                 df["MaritalStatus"].values,
                                 df["DistanceFromHome"].values,
                                 df["Education"].values,
                                 df["EducationField"].values,
                                 )))

datos_personal = convertir(datos_tabla_personal)

personal_id = list(df["IdEmployee"].values)

cursor.executemany(query_insertar_tablapersonal, datos_personal)

cnx.commit()


In [7]:
# Tabla EMPLOYEE_SATISFACTION

query_insertar_satisfaction = """ INSERT INTO `ABCCorporation`.`EMPLOYEE_SATISFACTION` (`EnvironmentSatisfaction`, `JobInvolvement`, `JobSatisfaction`, `RelationshipSatisfaction`, `WorkLifeBalance`) VALUES (%s, %s, %s, %s, %s)"""

datos_tabla_satisfaction = list(set(zip
                                (df["EnvironmentSatisfaction"].values,
                                 df["JobInvolvement"].values,
                                 df["JobSatisfaction"].values,
                                 df["RelationshipSatisfaction"].values,
                                 df["WorkLifeBalance"].values,
                                 )))

datos_satisfaction = convertir(datos_tabla_satisfaction)


cursor.executemany(query_insertar_satisfaction, datos_satisfaction)

satisfaction_id = cursor.lastrowid

cnx.commit()

In [8]:
# Tabla JOB_DATA

query_insertar_job_data = """ INSERT INTO `ABCCorporation`.`JOB_DATA` (`Attrition`, `JobLevel`, `JobRole`, `NumCompaniesWorked`, `TotalWorkingYears`, `YearsAtCompany`,`YearsSinceLastPromotion`, `YearsWithCurrManager`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""

datos_tabla_job_data = list(set(zip
                                (df["Attrition"].values,
                                 df["JobLevel"].values,
                                 df["JobRole"].values,
                                 df["NumCompaniesWorked"].values,
                                 df["TotalWorkingYears"].values,
                                 df["YearsAtCompany"].values,
                                 df["YearsSinceLastPromotion"].values,
                                 df["YearsWithCurrManager"].values,
                                 )))

datos_job_data = convertir(datos_tabla_job_data)


cursor.executemany(query_insertar_job_data, datos_job_data)

job_data_id = cursor.lastrowid

cnx.commit()

In [9]:
def extraer_datos(x):
    
    lista = []
    
    for dato in x:
        lista.append(cursor.lastrowid)
    
    return lista

In [12]:
# Tabla SALARY

query_insertar_salary = """ INSERT INTO `ABCCorporation`.`SALARY` (`HourlyRate`, `DailyRate`, `MonthlyRate`, `PercentSalaryHike`) VALUES (%s, %s, %s, %s)"""

datos_tabla_salary = list(set(zip
                                (df["HourlyRate"].values,
                                 df["DailyRate"].values,
                                 df["MonthlyRate"].values,
                                 df["PercentSalaryHike"].values
                                 )))

datos_salary = convertir(datos_tabla_salary)
salary_id = cursor.lastrowid  # Obtener el IdSalary generado

cursor.executemany(query_insertar_salary, datos_salary)


cnx.commit()

In [14]:
# Tabla JOB_EXTRAS

query_insertar_job_extras = """ INSERT INTO `ABCCorporation`.`JOB_EXTRAS` (`OverTime`, `BusinessTravel`, `StockOptionLevel`, `TrainingTimesLastYear`, `RemoteWork`, `PerformanceRating`) VALUES (%s, %s, %s, %s, %s, %s)"""

datos_tabla_job_extras = list(set(zip
                                (df["OverTime"].values,
                                 df["BusinessTravel"].values,
                                 df["StockOptionLevel"].values,
                                 df["TrainingTimesLastYear"].values,
                                 df["RemoteWork"].values,
                                 df["PerformanceRating"].values
                                 )))

datos_job_extras = convertir(datos_tabla_job_extras)
job_extras_id = cursor.lastrowid  # Obtener el IdJobExtras generado

cursor.executemany(query_insertar_job_extras, datos_job_extras)


cnx.commit()

In [ ]:
# Insertar datos en PERSONAL_INFO
query_personal = "INSERT INTO PERSONAL_INFO (EmployeeNumber, Age, DateBirth, Gender, MaritalStatus, DistanceFromHome, Education, EducationField) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
cursor.execute(query_personal, (employee_number, age, date_birth, gender, marital_status, distance, education, education_field))
personal_id = cursor.lastrowid  # Obtener el IdEmployee generado

# Insertar datos en EMPLOYEE_SATISFACTION
query_satisfaction = "INSERT INTO EMPLOYEE_SATISFACTION (EnvironmentSatisfaction, JobInvolvement, JobSatisfaction, RelationshipSatisfaction, WorkLifeBalance) VALUES (%s, %s, %s, %s, %s)"
cursor.execute(query_satisfaction, (env_satisfaction, job_involvement, job_satisfaction, relation_satisfaction, work_life_balance))
satisfaction_id = cursor.lastrowid  # Obtener el IdEmployeeSatisfaction generado

# Insertar datos en JOB_DATA
query_job_data = "INSERT INTO JOB_DATA (Attrition, JobLevel, JobRole, NumCompaniesWorked, TotalWorkingYears, YearsAtCompany, YearsSinceLastPromotion, YearsWithCurrManager) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
cursor.execute(query_job_data, (attrition, job_level, job_role, num_companies_worked, total_working_years, years_at_company, years_since_last_promotion, years_with_curr_manager))
job_data_id = cursor.lastrowid  # Obtener el IdJobData generado

# Insertar datos en SALARY

query_salary = "INSERT INTO SALARY (HourlyRate, DailyRate, MonthlyRate, PercentSalaryHike) VALUES (%s, %s, %s, %s)"
cursor.execute(query_salary, (hourly_rate, daily_rate, monthly_rate, percent_salary_hike))
salary_id = cursor.lastrowid  # Obtener el IdSalary generado

# Insertar datos en JOB_EXTRAS
query_job_extras = "INSERT INTO JOB_EXTRAS (OverTime, BusinessTravel, StockOptionLevel, TrainingTimesLastYear, RemoteWork, PerformanceRating) VALUES (%s, %s, %s, %s, %s, %s)"
cursor.execute(query_job_extras, (overtime, business_travel, stock_option_level, training_times, remote_work, performance_rating))
job_extras_id = cursor.lastrowid  # Obtener el IdJobExtras generado




In [ ]:
# Ahora insertar los datos en la tabla COMBINATION con los IDs obtenidos
query_combination = "INSERT INTO COMBINATION (IdEmployee, IdEmployeeSatisfaction, IdJobData, EmployeeSalary, IdJobExtras) VALUES (%s, %s, %s, %s, %s)"
cursor.execute(query_combination, (personal_id, satisfaction_id, job_data_id, salary_id, job_extras_id))

cnx.commit() 

In [ ]:
# Tabla COMBINATION

#query_insertar_combination = """ INSERT INTO `ABCCorporation`.`COMBINATION` (`IdEmployee`,`IdEmployeeSatisfaction`,`IdJobData`, `EmployeeSalary`, `IdJobExtras`) VALUES (%s, %s, %s, %s, %s)"""

#datos_tabla_combination = list(set(zip(
#    df["IdEmployee"].values,              # FK a PERSONAL_INFO (IdEmployee)
#    df["IdEmployeeSatisfaction"].values,  # FK a EMPLOYEE_SATISFACTION (IdEmployeeSatisfaction)
#    df["IdJobData"].values,               # FK a JOB_DATA (IdJobData)
#    df["EmployeeSalary"].values,          # FK a SALARY (EmployeeSalary)
#    df["IdJobExtras"].values              # FK a JOB_EXTRAS (IdJobExtras)
#)))

# Inserta los datos en la tabla COMBINATION
#cursor.executemany(query_insertar_combination, datos_tabla_combination)

# Confirma la transacción
#cnx.commit()